In [ ]:
!pip install q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! mkdir -p ~/.kaggle

In [ ]:
!cp  /content/drive/MyDrive/CSE-366/kaggle.json ~/.kaggle/kaggle.json

cp: cannot stat '/content/drive/MyDrive/CSE-366/kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download imbikramsaha/caltech-101

Dataset URL: https://www.kaggle.com/datasets/imbikramsaha/caltech-101
License(s): CC0-1.0
caltech-101.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!mkdir -p ./caltech

In [ ]:
!unzip caltech-101.zip -d ./caltech

Archive:  caltech-101.zip
replace ./caltech/caltech-101/BACKGROUND_Google/image_0001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from torchvision import datasets, transforms


transform = transforms.Compose([
transforms.Resize((128, 128)),
transforms.RandomHorizontalFlip(),
transforms.RandomVerticalFlip(),
transforms.RandomRotation(30),
transforms.ColorJitter(brightness=0.4, contrast=0.4,
saturation=0.4, hue=0.1),
transforms.RandomAffine(degrees=15, translate=(0.1, 0.1),
scale=(0.9, 1.1)),
transforms.RandomGrayscale(p=0.2),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,
0.224, 0.225])
])
dataset = datasets.ImageFolder(root='/content/caltech',
transform=transform)

In [ ]:
from torch.utils.data import random_split
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_data, val_data, test_data = random_split(dataset, [train_size,
val_size, test_size])

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

In [ ]:
from torchvision.models import vgg19
from torchvision.models import vgg19
import torch.nn as nn
model = vgg19(pretrained=True)
model.classifier[6] = nn.Linear(4096, 101)

In [ ]:
from torchvision.models import resnet50
model = resnet50(pretrained=True)

In [ ]:
from torchvision.models import resnet50
import torch.nn as nn

model = resnet50(pretrained=True)
model.fc = nn.Linear(2048, 101)


class_names = ['class1', 'class2', ..., 'class101']
print(class_names)

In [ ]:
from torchvision.models import efficientnet_b0
model = efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(1280, 101)
for 101 classes

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(10):
model.train()
for images, labels in train_loader:
optimizer.zero_grad()
outputs = model(images)
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()

In [ ]:
model.eval()
with torch.no_grad():
for images, labels in val_loader:
outputs = model(images)

In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {
'lr': [0.1, 0.01, 0.001],
'batch_size': [16, 32, 64]
}
best_params = None
best_accuracy = 0
for params in ParameterGrid(param_grid):
optimizer = optim.Adam(model.parameters(), lr=params['lr'])
train_loader = DataLoader(train_data,
batch_size=params['batch_size'], shuffle=True)
accuracy
print(f"Best Params: {best_params}")

In [ ]:
model.eval()

In [ ]:
with torch.no_grad():
for images, labels in test_loader:
outputs = model(images)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = []
y_true = []
with torch.no_grad():
for images, labels in test_loader:
outputs = model(images)
_, preds = torch.max(outputs, 1)
y_pred.extend(preds.numpy())
y_true.extend(labels.numpy())
cm = confusion_matrix(y_true, y_pred)
print(cm)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
def top_k_accuracy(output, target, k=5):
with torch.no_grad():
max_k_preds = torch.topk(output, k, dim=1).indices
correct = max_k_preds.eq(target.view(-1,
1).expand_as(max_k_preds))
return correct.any(dim=1).float().mean().item()

In [ ]:
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)
for i, acc in enumerate(per_class_accuracy):
print(f"Class {class_names[i]} Accuracy: {acc:.2f}")

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
features = []
labels_list = []
model.eval()
with torch.no_grad():
for images, labels in test_loader:
output = model(images)
features.append(output)
labels_list.append(labels)
features = torch.cat(features).numpy()
labels_list = torch.cat(labels_list).numpy()
tsne = TSNE(n_components=2, random_state=42)
reduced_features = tsne.fit_transform(features)
plt.scatter(reduced_features[:, 0], reduced_features[:, 1],
c=labels_list, cmap='tab10')
plt.colorbar()
plt.show()

In [ ]:
pip install grad-cam

In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
target_layer = model.layer4[-1]
architecture
cam = GradCAM(model=model, target_layer=target_layer)
for images, labels in test_loader:
grayscale_cam = cam(input_tensor=images,
target_category=labels[0])
cam_image = show_cam_on_image(images[0].permute(1, 2, 0).numpy(),
grayscale_cam)
plt.imshow(cam_image)
plt.show()